In [ ]:
class Attention(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.attention = nn.Linear(in_features, 1)

    def forward(self, x):
        weights = F.softmax(self.attention(x), dim=1)
        return (weights * x).sum(1)

class NewModel(nn.Module):
    def __init__(self, base_model, num_classes):
        super().__init__()
        self.base_model = base_model
        self.attention = Attention(base_model.fc.in_features)
        self.classifier = nn.Sequential(
            nn.Linear(base_model.fc.in_features, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.base_model(x)
        x = self.attention(x)
        return self.classifier(x)

# Load pre-trained ResNet-50
resnet = models.resnet50(weights='ResNet50_Weights.DEFAULT')
resnet.load_state_dict(torch.load('DDRresnet50.pth'))

# Create new model
num_classes = y_train_multi.shape[1]  # Assuming you have 5 classes
new_model = NewModel(resnet, num_classes)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(new_model.parameters(), lr=0.001)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
new_model.to(device)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    new_model.train()

    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = new_model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation
    new_model.eval()
    val_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in val_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = new_model(inputs)
            val_loss += criterion(outputs, labels).item()

            all_preds.append(outputs.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    val_loss /= len(val_dataloader)

    # Flatten predictions and labels for Quadratic Weighted Kappa calculation
    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    # Convert predictions to class labels
    pred_labels = np.round(all_preds).astype(int)
    true_labels = all_labels.astype(int)

    # Calculate Quadratic Weighted Kappa
    qwk = cohen_kappa_score(true_labels.sum(axis=1)-1, pred_labels.sum(axis=1)-1, weights='quadratic')

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}, QWK: {qwk:.4f}')

# Save the trained model
torch.save(new_model.state_dict(), 'DDRresnet50_with_attention.pth')

In [ ]:
from PIL import Image

# Load the image from the canvas
canvas_image = Image.open('canvas.png')

# Convert the image to grayscale
canvas_image = canvas_image.convert('L')

# Convert the image to a numpy array
canvas_array = np.array(canvas_image)

# Normalize the array to get weights in the range [0, 1]
weights = canvas_array / 255.0

In [51]:
import shutil
import os

# Define the class you are interested in
target_class = '4'

# Define the path to the new directory
new_dir = 'DDR-dataset/DR_grading/classes/4'

# Create the new directory if it doesn't exist
os.makedirs(new_dir, exist_ok=True)

# Open the text file and read the lines
with open('DDR-dataset/DR_grading/valid.txt', 'r') as f:
    lines = f.readlines()

# For each line in the file
for line in lines:
    # Split the line into image path and class label
    image_file, class_label = line.strip().split()

    # Prepend the directory path to the image file
    image_path = os.path.join('DDR-dataset/DR_grading/valid', image_file)

    # If the class label is the target class
    if class_label == target_class:
        # Define the path to the new file
        new_file_path = os.path.join(new_dir, os.path.basename(image_path))

        # Copy the image to the new directory
        shutil.copyfile(image_path, new_file_path)